In [1]:
!uv pip install -q langchain==0.2.16 matplotlib langchain-community==0.2.16 langchain-huggingface==0.0.3 langchain-text-splitters==0.2.4 chromadb==0.5.3 langchain-chroma==0.1.3 pandas ipywidgets

In [2]:
!uv pip install -q pyarrow

In [3]:
!uv pip install -q unidecode

In [4]:
import pandas as pd
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain_chroma import Chroma
from langchain.embeddings import OpenAIEmbeddings, OllamaEmbeddings
from langchain.document_loaders import TextLoader
from langchain.schema import Document, Generation, LLMResult
from langchain.llms import Ollama, BaseLLM
from langchain.chains import StuffDocumentsChain, RetrievalQA, LLMChain
from langchain_core.prompts import PromptTemplate
from langchain_community.llms import OpenAI
from langchain_text_splitters import CharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from pathlib import Path
import json
import requests
import re

In [5]:
import unidecode


In [6]:
from glob import glob

In [7]:
class LocalOllamaLLM(BaseLLM):
    api_url : str
    def _generate(self, prompt, stop):
        response = requests.post(f"{self.api_url}/api/generate", json={"model": "llama3.1", "prompt": str(prompt)})
        response.raise_for_status()
        response_text=''.join([json.loads(line)['response'] for line in response.text.splitlines()])
        generations=[]
        generations.append([Generation(text=response_text)])
        return LLMResult(generations=generations)


    def _llm_type(self):
        return "local"  # Or whatever type is appropriate for your local setup

llm = LocalOllamaLLM(api_url="http://127.0.0.1:11434")

embedder = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

system_prompt = (
    " Répondez à la question posée "
    " Utilisez le contexte (sélection des meilleurs paragraphes liés à la question) donné pour répondre à la question "
    " Répondez en bullet point comme ceci en copie-collant (sans reformulation) les phrases exactes issues du contexte qui t'ont permis de répondre. : "
    " ### Règles : "
    " * (règle 1) "
    " * (règle 2) "
    " Si la réponse ne se trouve pas dans le contexte, répondez par 'Null'"
    

    " Contexte : {context}  "
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, prompt)

Idées de system_prompt



 system_prompt_bis = (
    " Répondez à la question posée "
    " Utilisez le contexte (sélection des meilleurs paragraphes liés à la question) donné pour répondre à la question "
    "Répondez comme ceci :"
    "1. Un objet JSON avec les champs suivants :"
     " - contingent_annuel : nombre entier en jour par année (ou null s’il est non défini ou non déductible),"
     " - regle_majoration : true ou false selon s’il existe des règles de majoration des heures supplémentaires,"
     "- repos_compensateur_possible : true ou false selon que le paiement des heures supplémentaires peut être remplacé par un repos compensateur."
    "2. Ensuite, réécrire les phrases extraites du tete qui ont permi de répondre à ces questions"
    " Commencez impérativement ta réponse par la section JSON, puis ajoutez l’explication."
    " Si la réponse ne se trouve pas dans le contexte, répondez par 'Non'"
    " Contexte : {context}  




system_prompt_bis = (
    "Tu es un assistant intelligent chargé d'extraire une information dans un contexte donné. "
    "Ta tâche est de répondre à la question suivante : « À combien est fixé le contingent d’heures supplémentaires ? »\n\n"
    "Tu dois répondre uniquement par un JSON au format strict suivant :\n"
    "{{\n"
    '  "contingent_annuel": <valeur numérique entière ou null>\n'
    "}}\n\n"
    "- Si l’information n’est pas présente, mets la valeur `null` (sans guillemets).\n"
    "- Ne produis aucun commentaire, explication ou texte hors du JSON.\n\n"
    "Contexte : {context}"
)



system_prompt_bis = (
    "Tu es un assistant intelligent chargé d'extraire une information dans un contexte donné. "
    "Ta tâche est de répondre à la question suivante : « À combien est fixé le contingent d’heures supplémentaires ? »\n\n"
    "Tu dois répondre uniquement par un JSON au format strict suivant :\n"
    "{{\n"
    '  "contingent_annuel": <valeur numérique entière ou null>\n'
    "}}\n\n"
    "- Si l’information n’est pas présente, indique la valeur `null` (sans guillemets).\n"
    "- Ne produis aucun commentaire, explication ou texte hors du JSON.\n\n"
    "Attention : le **contingent d’heures supplémentaires** (également appelé **contingent annuel d’heures supplémentaires** ou parfois **quota d’heures supplémentaires**) "
    "désigne le **nombre maximal d’heures supplémentaires qu’un salarié est autorisé à effectuer sur une année**.\n"
    "Il **ne s’agit pas** du seuil hebdomadaire à partir duquel une heure devient supplémentaire.\n"
    "Par exemple, une phrase comme : « Constituent des heures supplémentaires celles effectuées au-delà de 37 heures » ne permet pas d’identifier le contingent annuel et doit conduire à renvoyer `null`.\n"
    " Mets `null` si l'information ne concerne **pas strictement** le contingent annuel d’heures supplémentaires.\n\n"
    "Contexte : {context}"
)


In [29]:
system_prompt_bis = (
    "Tu es un assistant intelligent chargé d'extraire une information dans un texte.\n"
    "Réponds uniquement par un JSON au format suivant :\n"
    "{{\n"
    '  "contingent_annuel": <valeur numérique entière ou null>\n'
    "}}\n\n"
    "Le contingent annuel d’heures supplémentaires est le volume d’heures supplémentaires autorisées pour un salarié sur une année.\n"
    "Ignore toute autre information comme le seuil de déclenchement des heures supplmémentaires, la durée annuelle de travail de référence, la durée légale de travail ou tout autre seuil de paiement des heures supplémentaires.\n"
    "Si l’information n’est pas clairement donnée, réponds avec null.\n\n"
    "Indication : tu dois particulièrement prêter attention aux formulations comme :\n"
    '  - "Le contingent annuel d\'heures supplémentaires est fixé à ..."\n'
    '  - "Le contingent est porté à ... "\n'
    '  - "Le volume d'heures supplémentaires autorisées est de ..." \n'
    "Contexte : {context}"
    "Si aucun contexte n’est fourni, réponds avec `null`.\n\n"
)

prompt_bis = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt_bis),
        ("human", "{input}"),
    ]
)
question_answer_chain_bis= create_stuff_documents_chain(llm, prompt_bis)

SyntaxError: unterminated string literal (detected at line 13) (1911938169.py, line 13)

In [9]:
df = pd.read_parquet("data/echantillon_1000_hs_2024_TOC.parquet")
df = df.rename(columns={"numdossier_new":"numdossier"})
df= df.set_index("numdossier")

In [10]:
df

,accorddocx,txt_trim_list,extracted_summary
numdossier,,,
T00724060165,\n\nACCORD COLLECTIF D’ENTREPRISE\n \nEntre \n...,"[, , ACCORD COLLECTIF D’ENTREPRISE, , Entre ,...","[ACCORD COLLECTIF D’ENTREPRISE, PREAMBULE,  ..."
T03324062903,\t\t\n\t\nACCORD D’ENTREPRISE RELATIF A L’ORGA...,"[\t\t, \t, ACCORD D’ENTREPRISE RELATIF A L’ORG...",[ACCORD D’ENTREPRISE RELATIF A L’ORGANISATION ...
T07424060533,AVENANT N°2 DE L’ACCORD COLLECTIF D’ENTREPRISE...,[AVENANT N°2 DE L’ACCORD COLLECTIF D’ENTREPRIS...,[AVENANT N°2 DE L’ACCORD COLLECTIF D’ENTREPRIS...
T03224060321,\n\nACCORD D’ENTREPRISE SUR LES HEURES SUPPLEM...,"[, , ACCORD D’ENTREPRISE SUR LES HEURES SUPPLE...",[ACCORD D’ENTREPRISE SUR LES HEURES SUPPLEMENT...
T06724061948,\naccord d’entreprise \nrelatif au contingent ...,"[, accord d’entreprise , relatif au contingent...","[accord d’entreprise , relatif au contingent d..."
...,...,...,...
T07424060477,ACCORD COLLECTIF INSTITUANT UN AMENAGEMENT DU ...,[ACCORD COLLECTIF INSTITUANT UN AMENAGEMENT DU...,[ACCORD COLLECTIF INSTITUANT UN AMENAGEMENT DU...
T06224062102,\t\nACCORD D’ENTREPRISE SUR L’AMENAGEMENT DU T...,"[\t, ACCORD D’ENTREPRISE SUR L’AMENAGEMENT DU ...",[ACCORD D’ENTREPRISE SUR L’AMENAGEMENT DU TEMP...
T05124061033,\n\n\nACCORD SUR LES MODALITES D’ORGANISATION ...,"[, , , ACCORD SUR LES MODALITES D’ORGANISATION...",[ACCORD SUR LES MODALITES D’ORGANISATION DU TE...


In [11]:
vector_store = Chroma(embedding_function=embedder,persist_directory="./chroma_db_article")

In [12]:
Q1= "À combien est fixé le contingent d’heures supplémentaires ? "

In [13]:
def search_and_invoke_llm(vector_store, numdossier, query, question_answer_chain, k=5):
    if k == 0:
        print(f"Échec de la recherche pour le dossier {numdossier}")
        return None

    try:
        retriever = vector_store.as_retriever(
            search_kwargs={
                "k": k,
                "filter": {"numdossier": str(numdossier)}
            }
        )
        
        retrieved_docs = retriever.invoke(query)
        chain = create_retrieval_chain(retriever, question_answer_chain)
        result = chain.invoke({"input": query})

        return {
            "answer": result.get("answer", ""),
            "context": [doc.page_content for doc in retrieved_docs]
        }

    except Exception as e:
        print(f"Erreur avec k={k}, tentative avec k={k-1} pour dossier {numdossier} : {e}")
        return search_and_invoke_llm(vector_store, numdossier, query, question_answer_chain, k=k-1)
        
def save_results(df, path, question_answer_chain): 
    Path(path).mkdir(exist_ok=True)
    already_done = {el.split("/")[1].split(".")[0] for el in glob(f"{path}/*.json")}

    for index, row in df.iterrows():
        if str(index) not in already_done:
            numdossier = str(index)

            result_json = {
                "num_dossier": numdossier,
                "responses": {},
                "context": {}
            }

            result = search_and_invoke_llm(vector_store, numdossier, Q1, question_answer_chain, k=2)

            if result:
                result_json["responses"][Q1] = result.get("answer", "")
                result_json["context"][Q1] = result.get("context", [])
            else:
                result_json["responses"][Q1] = None
                result_json["context"][Q1] = []

            with open(f"{path}/{numdossier}.json", "w", encoding="utf-8") as f:
                json.dump(result_json, f, indent=2, ensure_ascii=False)


In [14]:
!sudo apt update && sudo apt install --yes lshw

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease                 
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease               
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 https://ppa.launchpadcontent.net/git-core/ppa/ubuntu jammy InRelease
Hit:7 https://apt.postgresql.org/pub/repos/apt jammy-pgdg InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
55 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: https://apt.postgresql.org/pub/repos/apt/dists/jammy-pgdg/InRelease: Key is stored in legacy trusted.gpg keyring (/etc/apt/trusted.gpg), see the DEPRECATION section in apt-key(8) for details.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
lshw is already the newest

In [15]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> NVIDIA GPU installed.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [16]:
save_results(df,"results_llma_3_1_v1_2024_article",question_answer_chain_bis)

In [17]:
json_folder = "results_llma_3_1_v1_2024_article"
rows = []

# regex
contingent_pattern = re.compile(r'"contingent_annuel"\s*:\s*(\d+|null)')

for filepath in glob(f"{json_folder}/*.json"):
    with open(filepath, "r", encoding="utf-8") as f:
        data = json.load(f)
        num_dossier = data.get("num_dossier")

        # Récupération de la réponse brute
        response_text = data["responses"].get(
            "À combien est fixé le contingent d’heures supplémentaires ? "
        )

        contingent = None  # valeur par défaut

        if response_text:
            response_text = response_text.strip("` \n")
            match = contingent_pattern.search(response_text)
            if match:
                value = match.group(1)
                contingent = int(value) if value.isdigit() else None

        rows.append({
            "num_dossier": num_dossier,
            "contingent_annuel": contingent
        })

# Création et export CSV
df = pd.DataFrame(rows)
df.to_csv("contingents_annuels_article.csv", index=False)


In [18]:
results_llama3_1 = pd.read_csv("contingents_annuels_article.csv")
true_results= pd.read_csv("echantillon_eval_hs_2024_contingent_annote.csv")
results_llama3_1= results_llama3_1.rename(columns={"contingent_annuel":"contingent"})

In [19]:
true_results = true_results.rename(columns={"numdossier_new":"num_dossier"})

In [20]:
true_results

,num_dossier,contingent
0,T00724060165,NaN
1,T03324062903,400.0
2,T07424060533,NaN
3,T03224060321,NaN
4,T06724061948,360.0
...,...,...
95,T09224067019,NaN
96,T09224068199,NaN
97,T04524060557,220.0
98,T59L24063310,NaN


In [21]:
comparaison_df = pd.merge(
    results_llama3_1,
    true_results,
    on="num_dossier",
    how="inner",
    suffixes=('_llm', '_annot')  
)

In [22]:
comparaison_df

,num_dossier,contingent_llm,contingent_annot
0,T06924062286,200.0,200.0
1,T00124060638,180.0,180.0
2,T04324060351,290.0,290.0
3,T07424060700,500.0,500.0
4,T03624060237,350.0,350.0
...,...,...,...
95,T03424061818,NaN,180.0
96,T09424060951,300.0,300.0
97,T03724060392,250.0,250.0
98,T07424061457,400.0,400.0


In [23]:
comparaison_df["contingent_llm"] = comparaison_df["contingent_llm"].fillna("")
comparaison_df["contingent_annot"] = comparaison_df["contingent_annot"].fillna("")


In [24]:
comparaison_df[comparaison_df["contingent_llm"] != comparaison_df["contingent_annot"]]

,num_dossier,contingent_llm,contingent_annot
13,T06824061412,60.0,150.0
24,T05724060885,35.0,
26,T04524061125,200.0,
31,T03324061515,400.0,500.0
38,T02524061095,72.0,
44,T07524067789,,100.0
48,T05624061314,,265.0
51,T09424060886,,250.0
59,T05724061388,,300.0
73,T05624061324,156.0,


* T06824061412: "le paiement des heures supplémentaires est limité à 60 heures dans l’année"
  --> le contingent apparaît pourtant apparaît (même si pas si clairement)
* T05724060885	: durée légale de travail
* T04524061125 : "Le contingent annuel d’heures supplémentaires fixé par la loi est de 220 heures." --> 200 : invention
  --> parle d'augmentation mais rien n'est indiqué en plus
* T03324061515 : distinction de cas 400 et 500 pour d'autre (prendre le max ???)
* T02524061095 : ambiguïté dans la tournure de phrase
--> revoir cet exemple
* T07524067789 : mauvais découpage + discussion autour de la réelle valeure du contingent ici
  --> "Les heures supplémentaires effectuées au-delà du contingent annuel, pour les 100 premières heures, donnent droit à un repos compensateur dont la durée est égale à 50% des heures supplémentaires accomplies, soit 30 minutes pour chaque heure supplémentaire : 1 heure supplémentaire travaillée = 1,30 heure récupérée" n'apparaît pas
* T05624061314 : "Le contingent annuel d’heures supplémentaires, dénommé dans le présent accord contingent conventionnel, applicable aux salariés à temps complets couverts par le présent accord est fixé à 265 heures."
  --> 0"Heures complémentaires"
1"Décompte et rémunération des heures supplémentaires"    ne récupère que les titres --> sous-souspartie donc ne récupère que les titres (double saut d eligne) mais pas la partie en tant que telle
* T09424060886  la phrase apparaît très clairement "Le contingent annuel est fixé à 250 heures par an"
  prroblème : ce n'est pas sélectionné dans le retrieval car il y a 2 parties qui portent le titre d'heures supplémentaires et l'information sur le contingent annuel d'heures supp apparait dans la partie sur le repos compensateur de remplacement
  --> la structuration de l'accord n'est pas bonne 
  pareil ne récupère que les titres des parties et pas le cntenu (doucle saut de ligne)
* T05724061388	: revoir la manière dont ça découpe pcq à la première occurence du titre parfois ç correspond à juste une sous partie de l'article
  --> faire en sorte que ce soit successif ie à partir de la dernière partie on recherche encore le prochai titre mais on recommence pas la recherche depuis le début
* T05624061324 : " Les heures supplémentaires exceptionnelles seront celles effectuées au-delà de 156 heures sur le cycle de 4 semaines consécutives."
  --> ambiguï rediscuter : titre exceptionnel ??? parc eque au dessus du contingen ?? comme le défini la dsn ??
* T07424061214 : valeure normale au vue du retrieval --> revoir le découpage !!
  0"contingent annuel d’heures supplémentaires de 180 heures par an et par salarié. Ce contingent se révèle inadapté aux besoins et à l’activité de l’entreprise, notamment s’agissant des postes d’encadrement et de managers qui sont conduits à effectuer des heures supplémentaires de façon structurelle. Afin de faciliter la réalisation d’heures supplémentaires par les salariés et de permettre à la société de réagir de manière plus souple aux contraintes professionnelles et aux besoins de sa clientèle, tout en souhaitant préserver l’intérêt économique pour ses salariés, la société a souhaité augmenter le contingent annuel d’heures supplémentaires et définir le régime des heures supplémentaires effectuées à l’intérieur et au-delà de ce contingent. Pour autant, l’entreprise considère qu’une utilisation raisonnable de ce contingent doit rester une priorité. Le présent accord a été négocié dans le respect des dispositions des articles L. 2232-27 et suivants du Code du Travail. Il a été conclu sur le fondement de l’article L. 2232-23-1 du Code du travail relatif aux modalités de négociation dans les entreprises dont l’effectif habituel est compris entre 11 et moins de 50 salariés, dépourvues de délégués syndicaux. Les dispositions du présent accord, conclu en application des dispositions de l’article L. 3121-33 du Code du travail, reflètent la volonté des parties de trouver un équilibre entre l’intérêt général de l’entreprise et la collectivité de travail. IL A AINSI ÉTÉ CONVENU CE QUI SUIT"
1"Article 2 – Fixation du contingent annuel d’heures supplémentaire
* T00724060165:  "Constituent des heures supplémentaires les heures de travail effectif effectuées pour un salarié à temps plein au-delà de 35 heures en moyenne sur la période de référence"
  --> période de référence là encore
* T07524064067 : "les heures supplémentaires non structurelles sont celles comptabilisées au terme de la période annuelle de référence (soit au dernier jour du 12ème mois pour une période complète d’annualisation) au-delà de 1607 heures annuelles auxquelles sont ajoutées le heures supplémentaires structurelles"
  --> encore une fois récupère la période de référence
  --> tournure de phrase qui porte un peu à confusion
* T03424061818 : 0"Article 9.1. Contrepartie des heures accomplies au-delà du contingent"
1"ARTICLE 8 – RÉMUNÉRATION DES HEURES SUPPLÉMENTAIRES COMPRISES DANS LE CONTINGENT"
--> normal on a un retrieval avec uniquement les titres 

In [25]:
comparaison_df[comparaison_df["contingent_llm"] == comparaison_df["contingent_annot"]]

,num_dossier,contingent_llm,contingent_annot
0,T06924062286,200.0,200.0
1,T00124060638,180.0,180.0
2,T04324060351,290.0,290.0
3,T07424060700,500.0,500.0
4,T03624060237,350.0,350.0
...,...,...,...
94,T08424060522,300.0,300.0
96,T09424060951,300.0,300.0
97,T03724060392,250.0,250.0
98,T07424061457,400.0,400.0


In [27]:
from pprint import pprint

query = "À combien est fixé le contingent d’heures supplémentaires ?"
retriever = vector_store.as_retriever(search_kwargs={"k": 2, "filter": {"numdossier": "A02524003496"}})
results = retriever.invoke(query)

print("----- CONTEXTE RETOURNÉ -----")
for i, doc in enumerate(results):
    print(f"[{i}] {doc.metadata['titre']}")
    print(f"[{i}] {doc.metadata['id']}")
    print(doc.page_content[:300])
    print("---")


----- CONTEXTE RETOURNÉ -----
[0] Article 2. – Contingent d’heures supplémentaires  
[0] A02524003496_4
Article 2. – Contingent d’heures supplémentaires  

A compter du 1er mai 2024, le contingent d’heures supplémentaires applicable à l’ensemble des salariés de l’entreprise sera de 300 heures par an et par salarié. 
Il est rappelé que la période de référence pour calculer le contingent d’heures supplé
---
[1] Préambule 
[1] A02524003496_2
Préambule 

Le présent accord vise à définir et à fixer le contingent annuel d’heures supplémentaires applicable au sein de l’entreprise EPICERIE DU LAC, SARL. 
Les parties au présent accord ont en effet constaté que les dispositions en vigueur au sein de l’entreprise devaient être revues et adaptée
---
